In [1]:
import json
import os
from io import BytesIO

import eyed3
import ffmpeg
import requests
from yt_dlp import YoutubeDL

In [ ]:
# Playlist
youtube_urls = "https://music.youtube.com/playlist?list=PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R&si=wqnpJXsl19P_ufd5"

options = {
    "listformats": True,
    "ignoreerrors": True,
    # "cookiefile": "/home/lehoangchibach/Downloads/cookies.txt",
    "verbose": True,
}

with YoutubeDL(options) as ydl:
    info = ydl.extract_info(youtube_urls, download=False)

# with open("./outputs/data/playlists.json", "w") as f:
#     json.dump(info, f)

# with open("./outputs/data/playlists.json", "r") as f:
#     info = json.load(f)

[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2025.06.09 from yt-dlp/yt-dlp [339614a17] (pip) API
[debug] params: {'listformats': True, 'ignoreerrors': True, 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.18 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.11.9 (CPython x86_64 64bit) - Linux-6.15.5-100.fc41.x86_64-x86_64-with-glibc2.40 (OpenSSL 3.3.2 3 Sep 2024, glibc 2.40)
[debug] exe versions: ffmpeg 9c33b2f, ffprobe 9c33b2f
[debug] Optional libraries: brotli-1.1.0, certifi-2024.08.30, mutagen-1.47.0, requests-2.32.3, sqlite3-3.45.3, urllib3-2.2.3
[debug] Proxy map: {}
[debug] Request Handlers: urllib, requests
[debug] Plugin directo

[youtube:tab] Extracting URL: https://music.youtube.com/playlist?list=PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R&si=wqnpJXsl19P_ufd5


[youtube:tab] PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R: Downloading webpage


In [ ]:
len(info["entries"])

In [ ]:
options = {
    # "listformats": True,
    "cookiefile": "./outputs/cookies.txt",
    "outtmpl": "./outputs/downloads/%(title)s.%(ext)s",  # Output file name
}

for i, entry in enumerate(info["entries"]):
    if not entry:
        print(f"{i + 1}. None")
        continue

    formats = [f for f in entry["formats"] if f["resolution"] == "audio only"]
    formats.sort(key=lambda x: x["tbr"] or 0, reverse=True)
    format_id = formats[0]["format_id"]

    title = entry["title"]
    title = title.replace("｜", "|")
    title = title.replace("：", ":")
    title = title.replace("＂", "\"",)
    title = title.replace("/", "-")
    
    output_file = "./outputs/downloads/" + title + "." + formats[0]["audio_ext"]
    options["outtmpl"] = output_file
    
    if os.path.exists(output_file):
        print(f"{i + 1}. {entry['title']} - EXISTED")
        continue
    print(f"{i + 1}. {entry['title']}")

    options["format"] = format_id
    with YoutubeDL(options) as ydl:
        ydl.download(entry["original_url"])

print("Download completed!")

In [ ]:
not_exists = []
for i, entry in enumerate(info["entries"]):
    if not entry:
        print(f"{i + 1}. None")
        continue

    formats = [f for f in entry["formats"] if f["resolution"] == "audio only"]
    formats.sort(key=lambda x: x["tbr"] or 0, reverse=True)
    format_id = formats[0]["format_id"]

    title = entry["title"]
    title = title.replace("｜", "|")
    title = title.replace("：", ":")
    title = title.replace("＂", "\"",)
    title = title.replace("/", "-")

    fulltitle = entry["fulltitle"]
    fulltitle = fulltitle.replace("｜", "|")
    fulltitle = fulltitle.replace("：", ":")
    fulltitle = fulltitle.replace("＂", "\"",)
    fulltitle = fulltitle.replace("/", "-")

    webm = "./outputs/downloads/" + title + "." + formats[0]["audio_ext"]
    mp3 = "./outputs/thumbnails/" + fulltitle + "." + "mp3"
    # webm = webm.replace("|", "｜")
    # webm = webm.replace(":", "：")
    # webm = webm.replace("\"", "＂")
    # webm = webm.replace("/", "⧸")
    
    
    if not os.path.exists(webm):
        not_exists.append(webm)
        print(f"{i+1}. Not existed - ", webm)
        continue
    if os.path.exists(mp3):
        print(f"{i+1}. Existed - ", mp3)
        continue

    ffmpeg.input(webm).output(mp3, vn=None, acodec="libmp3lame", q=0).run()

    audiofile = eyed3.load(mp3)
    img_data = requests.get(entry["thumbnail"]).content
    audiofile.tag.images.set(3, img_data, "image/jpeg")
    audiofile.tag.artist = entry.get("artist", None)
    audiofile.tag.album = entry.get("album", None)
    audiofile.tag.title = entry.get("fulltitle", None)

    audiofile.tag.save()

    print(f"{i+1}. Thumbnail added to {mp3}")

In [ ]:
len(not_exists), not_exists